## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient
from pprint import pprint

# The project we already created URL
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
# Create a client
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# client.list_experiments()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1716552140527,
    'description': 'The NYC taxi predictor for trip duration',
    'last_updated_timestamp': 1716552269399,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1716552197476, current_stage='None', description='', last_updated_timestamp=1716552197476, name='nyc-taxi-regressor', run_id='29496d00ac1f41aebe59b1a05692b2a7', run_link='', source=('file:///D:/Learning/09. mlops/00. '
 'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/29496d00ac1f41aebe59b1a05692b2a7/artifacts/model'), status='READY', status_message=None, tags={'model': 'gradientboostingregressor', 'stage': 'staging'}, user_id=None, version=2>],
    'name': 'nyc-taxi-regressor',
    'tags': {}}


In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 94405a5e65324aeea527092b7badaf4b, rmse: 6.3184
run id: 3b0172e9350d4a3f9e6eb6434750bd48, rmse: 6.5997
run id: 8e2d6e3f34da45f3b4912125fbde7eb4, rmse: 6.6072
run id: 71b5329a5fa5433a9cb5b9b81e30650d, rmse: 6.7059
run id: 29496d00ac1f41aebe59b1a05692b2a7, rmse: 6.7423


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [70]:
run_id = "f3d3e0382aea4aeaa320da196df5a3a0"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716556747446, current_stage='None', description=None, last_updated_timestamp=1716556747446, name='nyc-taxi-regressor', run_id='f3d3e0382aea4aeaa320da196df5a3a0', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
 'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/f3d3e0382aea4aeaa320da196df5a3a0/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=6>

In [71]:
# Get the list of registry
model_name = "nyc-taxi-regressor"
# change from list_registered_model() to get_registered_model()
client.get_registered_model(name=model_name)

<RegisteredModel: aliases={'archived': 2, 'production': 3, 'staging': 1}, creation_timestamp=1716552140527, description='The NYC taxi predictor for trip duration', last_updated_timestamp=1716556747446, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716556747446, current_stage='None', description=None, last_updated_timestamp=1716556747446, name='nyc-taxi-regressor', run_id='f3d3e0382aea4aeaa320da196df5a3a0', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
 'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/f3d3e0382aea4aeaa320da196df5a3a0/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=6>,
 <ModelVersion: aliases=[], creation_timestamp=1716554445511, current_stage='Staging', description='The model version 3 was transitioned to staging on 2024-05-24', last_updated_timestamp=1716556147022, name='nyc-taxi-regressor', run_id='8e2d6e3f34da45f3b4912125fbde7eb4', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
 'Zoomcamp/

In [72]:
client.get_latest_versions(name=model_name)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11472\660683513.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1716556747446, current_stage='None', description=None, last_updated_timestamp=1716556747446, name='nyc-taxi-regressor', run_id='f3d3e0382aea4aeaa320da196df5a3a0', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
  'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/f3d3e0382aea4aeaa320da196df5a3a0/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=6>,
 <ModelVersion: aliases=[], creation_timestamp=1716554445511, current_stage='Staging', description='The model version 3 was transitioned to staging on 2024-05-24', last_updated_timestamp=1716556147022, name='nyc-taxi-regressor', run_id='8e2d6e3f34da45f3b4912125fbde7eb4', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
  'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/8e2d6e3f34da45f3b4912125fbde7eb4/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>,
 <ModelVersion: aliases=[], creation_timestamp=17165561421

In [73]:
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 6, stage: None
version: 3, stage: Staging
version: 5, stage: Production


C:\Users\Admin\AppData\Local\Temp\ipykernel_11472\232761966.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [74]:
model_version = 6
new_stage = "staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11472\3981096246.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716556747446, current_stage='Staging', description=None, last_updated_timestamp=1716556769329, name='nyc-taxi-regressor', run_id='f3d3e0382aea4aeaa320da196df5a3a0', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
 'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/f3d3e0382aea4aeaa320da196df5a3a0/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=6>

In [75]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716556747446, current_stage='Staging', description='The model version 6 was transitioned to staging on 2024-05-24', last_updated_timestamp=1716556773435, name='nyc-taxi-regressor', run_id='f3d3e0382aea4aeaa320da196df5a3a0', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
 'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/f3d3e0382aea4aeaa320da196df5a3a0/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=6>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [76]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [77]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [78]:
run_id

'f3d3e0382aea4aeaa320da196df5a3a0'

In [79]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'D:\\Learning\\09. mlops\\00. Zoomcamp\\mlops-zoomcamp\\02_Week_2\\preprocessor'

In [80]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

D:\Learning\09. mlops\00. Zoomcamp\mlops-zoomcamp\02_Week_2\venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [81]:
X_test = preprocess(df, dv)

In [82]:
target = "duration"
y_test = df[target].values

In [83]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

D:\Learning\09. mlops\00. Zoomcamp\mlops-zoomcamp\02_Week_2\venv\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: total: 12.1 s
Wall time: 16.9 s


D:\Learning\09. mlops\00. Zoomcamp\mlops-zoomcamp\02_Week_2\venv\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.8851658157505495}

In [84]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

D:\Learning\09. mlops\00. Zoomcamp\mlops-zoomcamp\02_Week_2\venv\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: total: 1min 15s
Wall time: 1min 41s


D:\Learning\09. mlops\00. Zoomcamp\mlops-zoomcamp\02_Week_2\venv\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.885624174675099}

In [85]:
client.transition_model_version_stage(
    name=model_name,
    version=6,
    stage="Production",
    archive_existing_versions=True
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11472\3256626065.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716556747446, current_stage='Production', description='The model version 6 was transitioned to staging on 2024-05-24', last_updated_timestamp=1716557324385, name='nyc-taxi-regressor', run_id='f3d3e0382aea4aeaa320da196df5a3a0', run_link=None, source=('file:///D:/Learning/09. mlops/00. '
 'Zoomcamp/mlops-zoomcamp/02_Week_2/mlruns/1/f3d3e0382aea4aeaa320da196df5a3a0/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=6>